In [1]:
import wandb
from robust_detection.wandb_config import ENTITY
from robust_detection.models.rcnn import RCNN
from robust_detection.data_utils.rcnn_data_utils import Objects_RCNN
#from torchmetrics.detection.map import MeanAveragePrecision
from torchmetrics.detection.mean_ap import MeanAveragePrecision
import pandas as pd
import pytorch_lightning as pl
import os
import torch
import numpy as np

wandb: Currently logged in as: moldenhof (robust-detection). Use `wandb login --relogin` to force relogin


In [2]:
gpu = 0
api = wandb.Api()

results = {}

sweep_dict = {"y9tovbu0":RCNN}
model_names = ["RCNN (Range DPL)"]


#data_dict = {"MMSynthetic":SyntheticMMDataModule, "Pendulum":PendulumDataModule, "CV":CVDataModule}
#data_dict = {"molecules/mol_labels/":Objects_RCNN}#, "mnist/alldigits_2":MNISTCountDataModule,  "mnist/alldigits_5":MNISTCountDataModule} #, \
            #"mnist/alldigits_large":MNISTCountDataModule, "mnist/alldigits_2_large":MNISTCountDataModule,  "mnist/alldigits_5_large":MNISTCountDataModule,}
#data_dict = {"mnist/alldigits_5/":Objects_RCNN}#, "mnist/alldigits_2":MNISTCountDataModule,  "mnist/alldigits_5":MNISTCountDataModule} #, \
            #"mnist/alldigits_large":MNISTCountDataModule
data_dict = {"clevr/clevr_skip_cube":Objects_RCNN}

fold_name = "fold"

In [3]:
df = pd.DataFrame()

In [4]:
for i_mod, sweep_name in enumerate(sweep_dict.keys()):

    pd_dict_acc = {"Model":model_names[i_mod] + " (Acc)"}
    pd_dict_map = {"Model":model_names[i_mod] + " (mAP)"}


    #model_cls = sweep_dict[sweep_names]
    #sweep_runs = []
    #for sweep_name in sweep_names:
    #    sweep_runs += api.sweep(f"{ENTITY}/object_detection/{sweep_name}").runs
    model_cls = sweep_dict[sweep_name]
    sweep = api.sweep(f"{ENTITY}/object_detection/{sweep_name}")
    sweep_runs = []
    sweep_runs += api.sweep(f"{ENTITY}/object_detection/{sweep_name}").runs
    print(sweep_runs)
    for ood in [False,True]:

        pd_dict_acc["Type"] = "OOD" if ood else "In-distribution"
        pd_dict_map["Type"] = "OOD" if ood else "In-distribution"

        
        for data_key in data_dict.keys():

            best_runs = []
            for fold in [0,1,2,3,4]:
                #runs_fold = [r for r in sweep_runs if (r.config.get(fold_name)==fold) and (r.config.get("target_data_path")==data_key)]
                runs_fold = [r for r in sweep_runs if (r.config.get(fold_name)==fold)]
                runs_fold_sorted = sorted(runs_fold,key = lambda run: run.summary.get("restored_val_acc"), reverse = True)
                best_runs.append(runs_fold_sorted[0])

            accuracies = []
            mAPs = []
            for run in best_runs:
                fname = [f.name for f in run.files() if "ckpt" in f.name][0]
                run.file(fname).download(replace = True, root = ".")
                model = model_cls.load_from_checkpoint(fname)
                os.remove(fname)

                hparams = dict(model.hparams)
                hparams["re_train"] = False
                hparams["data_path"]= data_key
                dataset = data_dict[data_key](**hparams)
                dataset.prepare_data()
                trainer = pl.Trainer(logger = False, gpus = 1)

                if ood:
                    preds = trainer.predict(model, dataset.test_ood_dataloader())
                else:
                    preds = trainer.predict(model, dataset.test_dataloader())
                
                Y = []
                Y_hat = []
                map_metric = MeanAveragePrecision()
                for pred in preds:
                    Y += pred["targets"]
                    Y_hat += pred["preds"]
                    
                    pred_map = [dict(boxes=pred["boxes"][i],scores=pred["scores"][i],labels=pred["preds"][i]) for i in range(len(pred["targets"]))]
                    target_map = [dict(boxes=pred["boxes_true"][i],labels=pred["targets"][i]) for i in range(len(pred["targets"]))]
                    map_metric.update(pred_map,target_map)
                
                mAP = map_metric.compute()
                accuracy = np.array([torch.equal(Y[i].sort()[0],Y_hat[i].sort()[0]) for i in range(len(Y))]).mean()
                print(accuracy)
                print(mAP)

                accuracies.append(accuracy)
                mAPs.append(mAP["map"])

            accuracies = np.array(accuracies)
            acc_mu = accuracies.mean()
            acc_std = accuracies.std()
            
            mAPs = np.array(mAPs)
            map_mu = mAPs.mean()
            map_std = mAPs.std()

            acc_str = "$ " + str(acc_mu.round(3))+ "\pm" +str(acc_std.round(3)) +" $"
            map_str = "$ " + str(map_mu.round(3))+ "\pm" +str(map_std.round(3)) +" $"


            pd_dict_acc[data_key] = acc_str
            pd_dict_acc[data_key + " mAP"] = map_str

            print(pd_dict_acc)

        df = df.append(pd_dict_acc,ignore_index =True)

[<Run robust-detection/object_detection/4syov2kr (finished)>, <Run robust-detection/object_detection/9kg0ubbn (finished)>, <Run robust-detection/object_detection/uusrvgm5 (finished)>, <Run robust-detection/object_detection/6ihgego5 (finished)>, <Run robust-detection/object_detection/ixpng3ru (finished)>]


/home/moldenho/miniconda3/envs/conda_poetry/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting: 0it [00:00, ?it/s]

0.996
{'map': tensor(0.9621), 'map_50': tensor(0.9988), 'map_75': tensor(0.9975), 'map_small': tensor(-1.), 'map_medium': tensor(0.9621), 'map_large': tensor(-1.), 'mar_1': tensor(0.8455), 'mar_10': tensor(0.9761), 'mar_100': tensor(0.9761), 'mar_small': tensor(-1.), 'mar_medium': tensor(0.9761), 'mar_large': tensor(-1.), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.)}


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


...

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting: 0it [00:00, ?it/s]

0.992
{'map': tensor(0.9379), 'map_50': tensor(0.9975), 'map_75': tensor(0.9975), 'map_small': tensor(-1.), 'map_medium': tensor(0.9379), 'map_large': tensor(-1.), 'mar_1': tensor(0.8314), 'mar_10': tensor(0.9605), 'mar_100': tensor(0.9605), 'mar_small': tensor(-1.), 'mar_medium': tensor(0.9605), 'mar_large': tensor(-1.), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.)}
{'Model': 'RCNN (Range DPL) (Acc)', 'Type': 'In-distribution', 'clevr/clevr_skip_cube': '$ 0.994\\pm0.001 $', 'clevr/clevr_skip_cube mAP': '$ 0.922\\pm0.035 $'}


/tmp/ipykernel_3587003/3439516838.py:87: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd_dict_acc,ignore_index =True)
/home/moldenho/miniconda3/envs/conda_poetry/lib/python3.8/site-packages/pytorch_lightning/loops/utilities.py:91: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


FileNotFoundError: [Errno 2] No such file or directory: '/home/moldenho/Projects/ProbKT/robust_detection/../generate_data/clevr/clevr_skip_cube/test_ood/images/'

In [5]:
print(df.loc[df.Model.str.contains("Acc")].to_latex(escape = False,index= False))

\begin{tabular}{llll}
\toprule
                 Model &            Type & clevr/clevr_skip_cube & clevr/clevr_skip_cube mAP \\
\midrule
RCNN (Range DPL) (Acc) & In-distribution &     $ 0.994\pm0.001 $ &         $ 0.922\pm0.035 $ \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3587003/3650743742.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.loc[df.Model.str.contains("Acc")].to_latex(escape = False,index= False))
